# Function calling with Langchain Agents, Tools

In this notebook we leveraage Langchain agents to perform function calling tasks. 

First we install the necessary python packages such as `duckduckgo-search`, `wikipedia`, and `xmltodict`, amongst others. 

- Initialize connection to **anthropic.claude-v2** using Bedrock
- Define and test the following tools individually
  - Wikipedia
  - Web Search and
  - PubMed search
  - Custom Weather tool that search for weather data using Latitude and Longitude
- Finally, we put this all together by initializing a Langchain agent executor

In [5]:
# !pip install -U duckduckgo-search wikipedia unstructured xmltodict --quiet

In [9]:
import ast
import json
import warnings

import boto3
from anthropic_bedrock import AI_PROMPT, HUMAN_PROMPT, AnthropicBedrock
from IPython.display import Markdown
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.llms.bedrock import Bedrock
from langchain.tools import DuckDuckGoSearchRun, PubmedQueryRun, tool
from langchain.utilities import WikipediaAPIWrapper
from rich import print
from utils import get_inference_parameters
import requests

%load_ext rich
warnings.filterwarnings("ignore")

The rich extension is already loaded. To reload it, use:
  %reload_ext rich


In [2]:
region = "us-west-2"
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime", region_name=region
)  # bedrock runtime boto3 client
bedrock_model_id = "anthropic.claude-v2"  # Amazon Bedrock Model ID for claude-v2

# anthropic claude model parameters
model_kwargs = get_inference_parameters("anthropic")

# print(model_kwargs)

# Define the llm with langchain.llms.bedrock
bedrock_llm = Bedrock(
    model_id=bedrock_model_id, client=bedrock_runtime, model_kwargs=model_kwargs
)

## Let's define a few tools

- [Wikipedia](https://wikipedia.org/) (Search wikipedia pages)
- Web Search (Search web using [DuckDuckGo](https://duckduckgo.com/) search engine)
- [PubMed](https://pubmed.ncbi.nlm.nih.gov/) (Medical papers and articles)
- Custom Weather Tool
  - Use `get_lat_long` and `get_weather` functions 

#### 01.Wikipedia Tool

[Wikipedia](https://wikipedia.org/) is a multilingual free online encyclopedia written and maintained by a community of volunteers, through open collaboration. Wikipedia is the largest and most-read reference work in history. 

>*NOTE:* To use this tool, install `wikipedia` package.

In [3]:
# Define the API wrapper for Wikipedia
wikipedia = WikipediaAPIWrapper()

# Define the Wikipedia tool with a description and the function to retrieve results from Wikipedia
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
)

# Test the defined Wikipedia tool
# Markdown(wikipedia_tool("What is the Archimedes Principle?"))

---

#### 02.DuckDuckGo Search Tool

[DuckDuckGo](https://duckduckgo.com/) is an internet privacy company most popularly known for their private search engine. The company emphasizes privacy and anonimity as one of the key principles behind all their products.

>*NOTE:* To use this tool, install `duckduckgo-search` package.

In [4]:
# Define the API wrapper for DuckDuckGo search
duckduckgo_search = DuckDuckGoSearchRun()

# Define the DuckDuckGo tool using a description and the function to retrieve results from DuckDuckGo search
duckduckgo_tool = Tool(
    name="DuckDuckGoSearch",
    func=duckduckgo_search.run,
    description="useful for when you need to answer questions about current events",
)

# Test the DuckDuckGo tool
# Markdown(duckduckgo_tool("Weather in Seattle, Washington?"))

---

#### 03.PubMed Tool

[PubMed](https://pubmed.ncbi.nlm.nih.gov/) is a free search engine for medical papers and articles.

>*NOTE:* To use this tool, install `xmltodict` package.

In [5]:
# Define the API wrapper for PubMed search
pubmed_search = PubmedQueryRun()

# Define the PubMed tool using a description and the function to retrieve results from PubMed
pubmed_tool = Tool(
    name="PubmedQueryRun",
    func=pubmed_search.run,
    description="Useful for when you need medical information",
)

# Test PubMed tool
# Markdown(pubmed_tool("Covid diagnosis"))

In [6]:
@tool("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns latitude and longitude for a given place name as a python dictionary object."""
    url = "https://nominatim.openstreetmap.org/search"

    params = {"q": place, "format": "json", "limit": 1}
    response = requests.get(url, params=params).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None


@tool("get_weather_dict")
def get_weather_dict(co_ord_str: str) -> dict:
    """
    Returns weather data for a given latitude and longitude as python dictionary.
    """
    if "{" in co_ord_str:
        co_ord_str = ast.literal_eval(co_ord_str)
        latitude = co_ord_str["latitude"]
        longitude = co_ord_str["longitude"]
    else:
        latitude = float(co_ord_str.split(",")[0].strip())
        longitude = float(co_ord_str.split(",")[1].strip())
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

## Putting it all together

- Define all the tools as a list
- Initialize the agent with tool list and `AgentType` of type `ZERO_SHOT_REACT_DESCRIPTION`
- Explicity tell the agent to handle parsing errors

In [12]:
# Define a list of tools for the agent
tools = [wikipedia_tool, duckduckgo_tool, pubmed_tool, get_lat_long, get_weather_dict]

# Initialize the agent with access to all the tooks in the list
react_agent = initialize_agent(
    tools,
    bedrock_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=4,
    handle_parsing_errors=True,
)

In [21]:
prompt_template = """

Human: Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [wikipedia, DuckDuckGo, PubmedQueryRun, get_lat_long, get_weather_dict]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

In [17]:
output = react_agent("Can you check the weather in Seattle for me?")
print(output)



> Entering new AgentExecutor chain...
 Here is the response in the requested format:

Question: Can you check the weather in Seattle for me?

Thought: To check the weather in a specific city, I need to find the latitude and longitude coordinates for that city first.

Action: get_lat_long
Action Input: Seattle
Observation: {'latitude': '47.6038321', 'longitude': '-122.330062'}
Thought: Now that I have the latitude and longitude for Seattle, I can use it to get the current weather data.

Action: get_weather_dict  
Action Input: {'latitude': '47.6038321', 'longitude': '-122.330062'}
Observation: {'latitude': 47.595562, 'longitude': -122.32442, 'generationtime_ms': 0.13208389282226562, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 40.0, 'current_weather_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature': '°C', 'windspeed': 'km/h', 'winddirection': '°', 'is_day': '', 'weathercode': 'wmo code'}, 'current_weather': {'time': '2024-01-0

{
    'input': 'Can you check the weather in Seattle for me?',
    'output': 'The current weather in Seattle is 3°C with 9 km/h winds from the SSE. The weathercode is 51 which 
indicates light drizzle.'
}

In [19]:
output = react_agent("What is the latest research on diabetes?")
print(output['output'])



> Entering new AgentExecutor chain...
 Here is an example response in the requested format:

Question: What is the latest research on diabetes?

Thought: To find the latest research on diabetes, I should search a medical research database like PubMed.

Action: pubmed_tool
Action Input: diabetes
Observation: pubmed_tool is not a valid tool, try one of [wikipedia, DuckDuckGoSearch, PubmedQueryRun, get_lat_long, get_weather_dict].
Thought: PubMed is a good source for medical research, so I will use the PubmedQueryRun tool.

Action: PubmedQueryRun
Action Input: diabetes
Observation: Published: --
Title: Antioxidant potential of chlorogenic acid in Age-Related eye diseases.
Copyright Information: © 2024 The Authors. Pharmacology Research & Perspectives published by British Pharmacological Society and American Society for Pharmacology and Experimental Therapeutics and John Wiley & Sons Ltd.
Summary::
Oxidative stress is an important mechanism of aging, and in turn, aging can also aggravate

Recent research on diabetes includes studies on the antioxidant chlorogenic acid for age-related eye diseases and 
telemedicine trials for blood pressure management in diabetes patients. The PubmedQueryRun tool provided summaries 
of two recent papers.

In [22]:
output = react_agent("Can you tell me about blackholes?")
print(output['output'])



> Entering new AgentExecutor chain...
 Here is an example response in the requested format:

Question: Can you tell me about blackholes?

Thought: I should look up information about blackholes on Wikipedia.

Action: wikipedia
Action Input: black hole
Observation: Page: Black hole
Summary: A black hole is a region of spacetime where gravity is so strong that nothing, including light and other electromagnetic waves, has enough energy to escape it. The theory of general relativity predicts that a sufficiently compact mass can deform spacetime to form a black hole. The boundary of no escape is called the event horizon. Although it has a great effect on the fate and circumstances of an object crossing it, it has no locally detectable features according to general relativity. In many ways, a black hole acts like an ideal black body, as it reflects no light. Moreover, quantum field theory in curved spacetime predicts that event horizons emit Hawking radiation, with the same spectrum as a bl

Agent stopped due to iteration limit or time limit.